In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
jdk_version = '11'
hadoop_version = 'hadoop2.7'

# Set Environment Variables
os.environ['SPARK_VERSION'] = spark_version
os.environ['JDK_VERSION'] = jdk_version
os.environ['HADOOP_VERSION'] = hadoop_version
os.environ['JAVA_HOME'] = f'/usr/lib/jvm/java-{jdk_version}-openjdk-amd64'
os.environ['SPARK_HOME'] = f'/content/{spark_version}-bin-{hadoop_version}'

# Update modules and install Spark and Java
!apt-get update
!apt-get install openjdk-$JDK_VERSION-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!tar xf $SPARK_VERSION-bin-$HADOOP_VERSION.tgz
!pip install -q findspark

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Level-Two-Starter-Code").getOrCreate()

# Load Amazon Data into Spark DataFrame

In [3]:
from pyspark import SparkFiles
raw_data_file = "amazon_reviews_us_Video_Games_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/" + raw_data_file
spark.sparkContext.addFile(url)
# Read and show the data
# Read and show the data
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [4]:
#  Viewing ALL CUSTOMERS on review date, customer, id, verified purchase, product title, star rating, helpful rating, total votes, and in vine program

analysis_df = df.select(["review_date","customer_id", "verified_purchase", "product_title","star_rating","helpful_votes","total_votes", "vine"])
analysis_df.show()

+-----------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-----------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
| 2015-08-31|   12039526|                Y|Thrustmaster T-Fl...|          5|            0|          0|   N|
| 2015-08-31|    9636577|                Y|Tonsee 6 buttons ...|          5|            0|          0|   N|
| 2015-08-31|    2331478|                Y|Hidden Mysteries:...|          1|            0|          1|   N|
| 2015-08-31|   52495923|                Y|GelTabz Performan...|          3|            0|          0|   N|
| 2015-08-31|   14533949|                Y|Zero Suit Samus a...|          4|            0|          0|   N|
| 2015-08-31|    2377552|                Y|Psyclone Recharge...|          1|            0|          0|   N|
| 2015-08-31|   17521011|   

In [5]:
# Summary statistics for ALL CUSTOMERS for star_rating, helpful votes, total votes
summary_analysis_df = analysis_df.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for ALL CUSTOMERS")
summary_analysis_df.show()

Summary statistics for ALL CUSTOMERS
+-------+------------------+------------------+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|
+-------+------------------+------------------+-----------------+
|  count|           1785997|           1785997|          1785997|
|   mean| 4.059892597803915|2.2593492598251843|3.757368573407458|
| stddev|1.3558117532896532| 19.17128509230589|22.45439170150866|
|    min|                 1|                 0|                0|
|    max|                 5|               999|              999|
+-------+------------------+------------------+-----------------+



In [6]:
# Ordering Star rating wth 5 being the highest for ALL CUSTOMERS

from pyspark.sql.functions import desc

star_rating_df = analysis_df.orderBy(analysis_df.star_rating.desc())
star_rating_df.show()

+-----------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
|review_date|customer_id|verified_purchase|       product_title|star_rating|helpful_votes|total_votes|vine|
+-----------+-----------+-----------------+--------------------+-----------+-------------+-----------+----+
| 2015-08-31|    2322846|                Y|Component HDTV AV...|          5|            0|          0|   N|
| 2015-08-31|   50165446|                Y|The Legend of Zel...|          5|            0|          0|   N|
| 2015-08-31|   48880662|                Y|Wireless Stereo H...|          5|            0|          0|   N|
| 2015-08-31|    9636577|                Y|Tonsee 6 buttons ...|          5|            0|          0|   N|
| 2015-08-31|   10712640|                Y|      Kingdom Hearts|          5|            0|          0|   N|
| 2015-08-31|    3014276|                Y|            Gameband|          5|            3|          7|   N|
| 2015-08-31|   29604302|   

# Filter by Votes

In [7]:
# Filter for greater than 20 total votes
from pyspark.sql.functions import col
cleaned_df = df.filter(col("total_votes")  >=20)
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    7142190| R4PKAZRQJJX14|B00QZLVCU0|     210935604|Valve - DOTA 2 St...|     Video Games|          1|           21|         34|   N|                N|What store doesn'...|Who pays 4 dollar...| 2015-08-31|
|         US|    1085641|R2CI0Y288CC7E2|B00RHI62GY|     626589765|ONE PIECE Pirate ...|     Video Games|          1|    

In [8]:
# Filter for greater than 50% helpful vote percentage
cleaned_df = df.filter(col("helpful_votes")  >=50)
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34170274|R127WEQY2FM1T3|B00W435BU0|     487294121|       Madden NFL 16|     Video Games|          1|          147|        175|   N|                Y|Hot Mess. Don't Buy.|This is an absolu...| 2015-08-31|
|         US|   37932477|R2FJ94555FZH32|B00WJ0Z194|     116945357|Infinity 3.0 Star...|     Video Games|          2|    

# Describe Stats

In [17]:
# Paid reviews
from pyspark.sql.functions import col, avg
paid_df = cleaned_df.filter(cleaned_df['vine']== 'Y')
paid_df.describe().show(5)

+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|product_id|     product_parent|       product_title|product_category|       star_rating|     helpful_votes|      total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+------------------+------------------+-----------------+----+-----------------+--------------------+--------------------+-----------+
|  count|         27|                  27|            27|        27|                 27|                  27|              27|                27|                27|               27|  27|               27|               

In [18]:
# Summary statistics for VINE CUSTOMERS for star_rating, helpful votes, total votes

paid_analysis_df = paid_df.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for VINE CUSTOMERS")
paid_analysis_df.show()

Summary statistics for VINE CUSTOMERS
+-------+------------------+------------------+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|
+-------+------------------+------------------+-----------------+
|  count|                27|                27|               27|
|   mean| 4.407407407407407|120.33333333333333|131.8148148148148|
| stddev|0.6360490582953523| 93.39247210644895|97.70719091599736|
|    min|                 3|               111|              102|
|    max|                 5|                97|               88|
+-------+------------------+------------------+-----------------+



In [19]:
# Top 20 rated products with the Top helpful votes for VINE CUSTOMERS

from pyspark.sql.functions import desc

vine_helpful_votes_df = paid_analysis_df.orderBy(paid_analysis_df.helpful_votes.desc())
vine_helpful_votes_df = vine_helpful_votes_df.filter('star_rating = 5')

print("Top 20 rated products with the Top helpful votes for VINE CUSTOMERS")
vine_helpful_votes_df.show()

Top 20 rated products with the Top helpful votes for VINE CUSTOMERS
+-------+-----------+-------------+-----------+
|summary|star_rating|helpful_votes|total_votes|
+-------+-----------+-------------+-----------+
|    max|          5|           97|         88|
+-------+-----------+-------------+-----------+



In [22]:
# Unpaid reviews
unpaid_df = cleaned_df.filter(cleaned_df['vine']== 'N')
unpaid_df.describe().show(5)

+-------+-----------+--------------------+--------------+------------------+-------------------+--------------------+----------------+-----------------+------------------+------------------+----+-----------------+--------------------+--------------------+-----------+
|summary|marketplace|         customer_id|     review_id|        product_id|     product_parent|       product_title|product_category|      star_rating|     helpful_votes|       total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-------+-----------+--------------------+--------------+------------------+-------------------+--------------------+----------------+-----------------+------------------+------------------+----+-----------------+--------------------+--------------------+-----------+
|  count|       8734|                8734|          8734|              8734|               8734|                8734|            8734|             8734|              8734|              8734|8734| 

In [23]:
# Summary statistics for NON-VINE CUSTOMERS for star_rating, helpful votes, total votes

unpaid_summary_analysis_df = unpaid_df.select(["star_rating","helpful_votes","total_votes"]).describe()

print("Summary statistics for NON-VINE CUSTOMERS")
unpaid_summary_analysis_df.show()

Summary statistics for NON-VINE CUSTOMERS
+-------+-----------------+------------------+------------------+
|summary|      star_rating|     helpful_votes|       total_votes|
+-------+-----------------+------------------+------------------+
|  count|             8734|              8734|              8734|
|   mean|3.637737577284177|129.16487291046485|147.67643691321274|
| stddev|1.555779983254492|235.01549015292247| 254.5195863016123|
|    min|                1|               100|               100|
|    max|                5|               999|               999|
+-------+-----------------+------------------+------------------+



In [24]:
# Top 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS

from pyspark.sql.functions import desc

non_vine_helpful_votes_df = unpaid_summary_analysis_df.orderBy(unpaid_summary_analysis_df.helpful_votes.desc())
non_vine_helpful_votes_df = non_vine_helpful_votes_df.filter('star_rating = 5')

print("Top 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS")
non_vine_helpful_votes_df.show(20)

Top 20 rated products with the Top helpful votes for NON-VINE CUSTOMERS
+-------+-----------+-------------+-----------+
|summary|star_rating|helpful_votes|total_votes|
+-------+-----------+-------------+-----------+
|    max|          5|          999|        999|
+-------+-----------+-------------+-----------+



# Determine the percentage of five-star review among vine reviews

In [25]:
# CODE HERE
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)

print(f'Number of paid reviews {paid_number}')
print(f'Number of paid five star reviews {paid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_vine * 100}%')

Number of paid reviews 27
Number of paid five star reviews 13
Percantage of paid reviews that are five stars 48.148148148148145%


# Determine the percentage of five-star review among non-Vine reviews

In [26]:
# CODE HERE
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)

print(f'Number of unpaid reviews {unpaid_number}')
print(f'Number of unpaid five star reviews {unpaid_five_star_number}')
print(f'Percantage of paid reviews that are five stars {percentage_five_star_non_vine * 100}%')

Number of unpaid reviews 8734
Number of unpaid five star reviews 3942
Percantage of paid reviews that are five stars 45.13395923975269%


In [29]:
#Count of Vine Customers and Non-Vine Customers

In [31]:
vine_customers_df  = paid_df.groupby("vine").agg({"vine":"count"})
vine_customers_df.show()

+----+-----------+
|vine|count(vine)|
+----+-----------+
|   Y|         27|
+----+-----------+



In [32]:
#  Count of NON-VINE CUSTOMERS

non_vine_customers_df = unpaid_df.groupby("vine").agg({"vine":"count"})
non_vine_customers_df.show()

+----+-----------+
|vine|count(vine)|
+----+-----------+
|   N|       8734|
+----+-----------+



In [33]:
# Combine customer count 
# Source link: https://sparkbyexamples.com/spark/spark-dataframe-union-and-union-all/

total_customers_df = vine_customers_df.unionAll(non_vine_customers_df)
total_customers_df = total_customers_df.withColumnRenamed('count(vine)', 'Number of customers')

print("Number of Vine and Non-Vine customers")
total_customers_df.show()

Number of Vine and Non-Vine customers
+----+-------------------+
|vine|Number of customers|
+----+-------------------+
|   Y|                 27|
|   N|               8734|
+----+-------------------+



In [34]:
#Number of Reviews made by the top rated and most helpful VINE CUSTOMERS